In [ ]:
import cv2
import pydicom
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [ ]:
def normal_window(data, window_center, window_width, intercept, slope):
    """
    Extract simple image window using given parameters.
    https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing
    :return: A single window extracted from given data
    """
    data = (data * slope + intercept)
    data_min = window_center - window_width // 2
    data_max = window_center + window_width // 2
    data[data < data_min] = data_min
    data[data > data_max] = data_max
    return data


def sigmoid_window(data, window_center, window_width, intercept, slope, U=1.0, eps=(1.0 / 255.0)):
    """
    Extract image window from data using given parameters and rescale values into sigmoid.
    https://www.kaggle.com/reppic/gradient-sigmoid-windowing
    :return: A single window extracted from given data where min value is 0 and max value is 1
    """
    data = (data * slope + intercept)
    ue = np.log((U / eps) - 1.0)
    W = (2 / window_width) * ue
    b = ((-2 * window_center) / window_width) * ue
    z = W * data + b
    data = U / (1 + np.power(np.e, -1.0 * z))
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return data


def get_first_of_dicom_field_as_int(x):
    """
    Get value x as int or extract first value from x if it is pydicom MultiValue
    """
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)


def get_window_metadata(data):
    """
    Get parameters needed for correct windowing from dicom metadata
    """
    intercept = data.RescaleIntercept
    slope = data.RescaleSlope
    window_center = get_first_of_dicom_field_as_int(data.WindowCenter)
    window_width = get_first_of_dicom_field_as_int(data.WindowWidth)

    return [window_center, window_width, intercept, slope]


def normalize(data):
    """
    Rescale data into values 0 - 255 and return it as int16
    """
    data = np.float32(data)
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    data *= 255
    return np.int16(data)


def load_and_preprocess(path, img_id, img_sz, window_type, window_conf):
    """
    Load data from dicom file and create image where RGB layers are used to store different window information. Final
    image is normalized to values 0-255, cast as int16 and resized to img_sz.
    :param path:
    :param img_id:
    :param img_sz:
    :param window_type: 0 - normal_window, 1-sigmoid window
    :param window_conf: 2d array (shape: (3,2)) with information about window center and width. -1 indicates that value
     should be load from dicom metadata. Example: [[-1, -1], [80, 200], [600, 2800]]
    :return: image as int16 3d array
    """
    data = pydicom.dcmread(path + img_id + '.dcm')
    window_center_metadata, window_width_metadata, intercept, slope = get_window_metadata(data)
    data = data.pixel_array

    img = np.zeros((*np.shape(data), len(window_conf)), dtype=np.int16)

    for i, window_data in enumerate(window_conf):
        window_center, window_width = window_data

        if window_width == -1:
            window_center = window_center_metadata
            window_width = window_width_metadata

        if window_type == 0:
            layer = normal_window(data, window_center, window_width, intercept, slope)
        else:
            layer = sigmoid_window(data, window_center, window_width, intercept, slope)

        img[..., i] = normalize(layer)

    if np.shape(img)[2] == 1:
        img = np.squeeze(img)
        img = np.repeat(img[..., None], 3, 2)

    if img_sz:
        img = cv2.resize(img, (img_sz, img_sz))

    return img


def load_dcm_data(path, img_id):
    """
    Load pixel array from dmc file with given image ID.
    """
    data = pydicom.dcmread(path + img_id + '.dcm')
    data = data.pixel_array
    return data


def load_and_preprocess_adj(path, img_id, img_sz, meta):
    """
    Load data from dicom file and create image where RGB layers are used to store current and adjacent slices. If there
    is no adjacent slice available the current slice is used instead. Final image contains full data (no window is used)
    and is not normalized (contains full information from dicom.pixel_array).
    :param path:
    :param img_id:
    :param img_sz:
    :param meta: metadata containing Image ID, PatientID, StudyInstanceUID ordered by ImagePositionPatient[2]
    :return: image as float32 3d array
    """
    data = load_dcm_data(path, img_id)
    img = np.zeros((*np.shape(data), 3), dtype=np.float32)

    img_meta = meta[meta['Image'] == img_id]
    patient_id = img_meta['PatientID'].values[0]
    sequence_id = img_meta['StudyInstanceUID'].values[0]
    seq_meta = meta[(meta['PatientID'] == patient_id) & (meta['StudyInstanceUID'] == sequence_id)]
    seq_meta.reset_index(drop=True, inplace=True)

    img_in_seg_meta = seq_meta[seq_meta['Image'] == img_id]
    img_in_seg_id = img_in_seg_meta.index.to_numpy()

    if img_in_seg_id == 0:
        img[..., 0] = data
    else:
        img[..., 0] = load_dcm_data(path, seq_meta.loc[img_in_seg_id - 1, 'Image'].values[0])

    img[..., 1] = data

    if img_in_seg_id == np.max(seq_meta.index.to_numpy()):
        img[..., 2] = data
    else:
        img[..., 2] = load_dcm_data(path, seq_meta.loc[img_in_seg_id + 1, 'Image'].values[0])

    if img_sz:
        img = cv2.resize(img, (img_sz, img_sz))

    return img

In [ ]:
def load_csv(csv_path):
    data_df = pd.read_csv(csv_path)
    data_df[['ID', 'Image', 'Diagnosis']] = data_df['ID'].str.split('_', expand=True)
    data_df = data_df.drop(['ID'], axis=1).drop_duplicates()
    data_df = data_df.pivot(index='Image', columns='Diagnosis', values='Label').reset_index()
    data_df['Image'] = 'ID_' + data_df['Image']
    return data_df

def get_metadata(path, files):
    train_di = {}

    for filename in tqdm(files):
        dcm = pydicom.dcmread(path + filename)
        all_keywords = dcm.dir()
        ignored = ['Rows', 'Columns', 'PixelData']

        if 'ID' not in train_di:
            train_di['ID'] = []
            
        train_di['ID'].append(filename[:-4])
        
        for name in all_keywords:
            if name in ignored:
                continue

            if name not in train_di:
                train_di[name] = []

            train_di[name].append(dcm[name].value)

    df = pd.DataFrame(train_di)
    
    return df

In [ ]:
data_folder = '/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'

In [ ]:
train_df = load_csv(data_folder + 'stage_2_train.csv')

In [ ]:
index = 78431
data_path = data_folder + 'stage_2_train/'
img_id = train_df.loc[index, 'Image']
print(train_df.loc[index])

In [ ]:
img_sz = 378
window_conf = [[-1, -1], [80, 200], [600, 2800]]
img_type0 = load_and_preprocess(data_path, img_id, img_sz, 0, window_conf)
img_type1 = load_and_preprocess(data_path, img_id, img_sz, 1, window_conf)

In [ ]:
def set_ax_properties(ax):
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False) 
    ax.tick_params(axis='both', which='both', length=0)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
img_dcm = load_dcm_data(data_path, img_id)
ax.imshow(img_dcm, cmap='bone')
set_ax_properties(ax)
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,8))
ax = fig.subplots(nrows=2, ncols=3)

for i in range(0,3):
    ax[0][i].imshow(img_type0[...,i])
    ax[1][i].imshow(img_type1[...,i])
    set_ax_properties(ax[0][i])
    set_ax_properties(ax[1][i])
    
ax[0][0].set_ylabel('Normal window')
ax[1][0].set_ylabel('Sigmoid window')